# Onboarding as a Data Scientist

This notebook is the Data Scientist view of part 5. You should have already run through steps 1-7 in the OM Authority notebook 5. If you have not done so proceed to port 8888 (either localhost or aws) and complete the steps in notebook `5. Onboarding a Data Scientist` first.



## 8. Initialising the Data Scientist Controller

This is very similar to the code used for the OM Authority accept the arguments used to configure the AriesAgentController are different.

The point to the ports and webhook location used when starting the data-scientist-agent ACA-Py instance. You can see these arguments in the `manage` script line 145.


In [1]:
%autoawait
import time
import asyncio
from termcolor import colored,cprint

from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8062
WEBHOOK_BASE = ""
ADMIN_URL = "http://datascientist-agent:8061"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
response = await agent_controller.wallet.get_public_did()
print(response)

{'result': {'did': 'Gy1EvbGAMsgscNH1rAvuu3', 'verkey': '9hhpMCb1n6ecjzH21cF1mMNX8Z67V47A51UwVmBKCXtz', 'posture': 'public'}}


In [3]:
response = await agent_controller.wallet.get_did_endpoint('Gy1EvbGAMsgscNH1rAvuu3')

In [4]:
print(response)

{'did': 'Gy1EvbGAMsgscNH1rAvuu3', 'endpoint': None}


## 9. Start Webhook Server and Register Listeners

We register listeners for the connections and issue-credential topic as before. However, the output will be slightly different because the Data Scientist is on the other side of the protocol to the OM Authority. We could not see any webhooks from this side before because they were managed by the mobile application you used.

In [2]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())


def cred_handler(payload):
    print("Handle Credential Webhook Payload")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange ID {exchange_id}")
    print("Agent Protocol Role", role)
    print("Protocol State ", state )
    if state == "offer_recieved" and role == "holder":
        print(f"You are being offered a credential with the following: {attributes}")
    if state == "credential_acked":
        print("Your agent has stored the credential in it's wallet")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

def connections_handler(payload):
    print("Handle Connection Webhook Payload")
    connection_id = payload["connection_id"]
    print("Connection ID", connection_id)

    state = payload['state']
    print("State", state)
    if state == 'active':
#         print('Connection {0} changed state to active'.format(connection_id))
        print(colored("Connection {0} changed state to active".format(connection_id), "red", attrs=["bold"]))


connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

agent_controller.register_listeners([cred_listener,connection_listener], defaults=True)

## 10. Copy Invitation from OM Authority

You should have **copied** the invitation object output from step 7. in the OM Authority notebook. **First** paste that over the example object shown below, **then** run the cell.

In [3]:
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'e085a70e-75b1-4be0-91c1-f79fb4dbe75b', 'label': 'OM_AUTHORITY', 'serviceEndpoint': 'https://0bd98e2df434.ngrok.io', 'recipientKeys': ['BrTQc7R4QpZsfBEoc3g7H5yX8TiRgYR5w35qodnbphqD']}

## 11. Accept the Connection

Pay attention to the webhook messages that get output as the datascientist-agent exchanges messages with the om-authority. First messages are from the connections topic handler and then the issue-credential handler.

In this instance the Data Scientist agent is configured to accept any credentials it gets offered through a flag passed into the `aca-py start` command.

The flags set on all agents in this notebook series can be found at `PyDentity/configuration/aries-args-advanced.yaml`

They are:

* auto-accept-requests: true
* auto-respond-credential-proposal: true
* auto-respond-credential-offer: true
* auto-respond-credential-request: true
* auto-store-credential: true
* auto-respond-presentation-proposal: true
* auto-respond-presentation-request: true
* preserve-exchange-records: true
* debug-connections: true
* debug-credentials: true
* debug-presentations: true
* enable-undelivered-queue: true


In [4]:
await agent_controller.connections.accept_connection(invitation)

Handle Connection Webhook Payload
Connection ID c6c60a8d-4f62-4e2b-9ed9-220f3dcb6a7d
State invitation
Handle Connection Webhook Payload
Connection ID c6c60a8d-4f62-4e2b-9ed9-220f3dcb6a7d
State request


{'invitation_mode': 'once',
 'routing_state': 'none',
 'invitation_key': 'BrTQc7R4QpZsfBEoc3g7H5yX8TiRgYR5w35qodnbphqD',
 'my_did': 'SwmW6XVrYwwj3qeWPxiAgr',
 'connection_id': 'c6c60a8d-4f62-4e2b-9ed9-220f3dcb6a7d',
 'request_id': '111e2d2b-bf67-4b0b-bad2-7c5675d4d19d',
 'accept': 'manual',
 'updated_at': '2021-03-01 16:45:07.108165Z',
 'their_label': 'OM_AUTHORITY',
 'initiator': 'external',
 'created_at': '2021-03-01 16:45:07.083437Z',
 'state': 'request'}

Handle Connection Webhook Payload
Connection ID c6c60a8d-4f62-4e2b-9ed9-220f3dcb6a7d
State response
Handle Connection Webhook Payload
Connection ID c6c60a8d-4f62-4e2b-9ed9-220f3dcb6a7d
State active
Connection c6c60a8d-4f62-4e2b-9ed9-220f3dcb6a7d changed state to active
Handle Credential Webhook Payload
Credential exchange ID d454167d-1014-42c0-9a22-6f4c0bc28f43
Agent Protocol Role holder
Protocol State  offer_received
Handle Credential Webhook Payload
Credential exchange ID d454167d-1014-42c0-9a22-6f4c0bc28f43
Agent Protocol Role holder
Protocol State  request_sent
Handle Credential Webhook Payload
Credential exchange ID d454167d-1014-42c0-9a22-6f4c0bc28f43
Agent Protocol Role holder
Protocol State  credential_received
Handle Credential Webhook Payload
Credential exchange ID d454167d-1014-42c0-9a22-6f4c0bc28f43
Agent Protocol Role holder
Protocol State  credential_acked
Your agent has stored the credential in it's wallet


## 12. Review the output from the webhook listeners in the OM Authority notebook

The state from the issue-credential webhook payloads should be slightly different.

Don't forget to terminate the AriesAgentController in this tutorial

In [5]:
await agent_controller.terminate()

## Continue to Notebook 6.

Remember to start in the OM Authority jupyter notebook. We will not be interacting with the data scientist for this set of notebooks.